In [22]:
import cv2
import numpy as np
import torch
import time
import threading
import pyttsx3
from ultralytics import YOLO
from playsound import playsound
from collections import deque

# Load YOLOv8 model for object detection
model = YOLO("yolov8n.pt")

# Initialize text-to-speech (TTS)
tts_engine = pyttsx3.init()
tts_engine.setProperty('rate', 160)

# Assign random colors to each class
np.random.seed(42)
colors = {cls_id: tuple(np.random.randint(0, 255, 3).tolist()) for cls_id in range(80)}

# Sound file path
alert_sound = "alert.wav"

# Object tracking memory (last N detections)
object_history = deque(maxlen=10)

def speak(text):
    """Speak text asynchronously using pyttsx3"""
    def speak_thread():
        tts_engine.say(text)
        tts_engine.runAndWait()

    threading.Thread(target=speak_thread, daemon=True).start()

def play_alert_sound():
    """Play alert sound using playsound in a separate thread"""
    def sound_thread():
        try:
            playsound(alert_sound)
        except Exception as e:
            print(f"Error playing sound: {e}")

    threading.Thread(target=sound_thread, daemon=True).start()

def detect_objects(frame):
    """Detect objects using YOLOv8"""
    results = model(frame)
    detected_objects = set()

    for result in results:
        if hasattr(result, "boxes") and result.boxes is not None:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                conf = box.conf[0].item()
                cls = int(box.cls[0].item())
                class_name = model.names.get(cls, 'Unknown')

                detected_objects.add(class_name)
                color = colors.get(cls, (0, 255, 0))

                # Draw bounding box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                label = f"{class_name} {conf:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

                # Traffic Sign Detection
                if class_name in ["stop sign", "traffic light", "speed limit"]:
                    speak(f"Detected {class_name}")

    # Avoid duplicate alerts
    if detected_objects and detected_objects not in object_history:
        object_history.append(detected_objects)
        speak(f"Detected: {', '.join(detected_objects)}")

    return frame

def detect_lanes(frame):
    """Lane detection using Canny Edge and Hough Transform"""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)

    height, width = frame.shape[:2]
    mask = np.zeros_like(edges)
    region_of_interest = np.array([[(50, height), (width // 2 - 50, height // 2), (width // 2 + 50, height // 2), (width - 50, height)]], np.int32)
    cv2.fillPoly(mask, region_of_interest, 255)
    
    masked_edges = cv2.bitwise_and(edges, mask)

    lines = cv2.HoughLinesP(masked_edges, 2, np.pi / 180, 100, np.array([]), minLineLength=50, maxLineGap=150)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

    return frame

# Open webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not access webcam.")
else:
    prev_time = time.time()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Measure FPS
        curr_time = time.time()
        fps = 1 / (curr_time - prev_time)
        prev_time = curr_time

        # Detect objects and lanes
        frame = detect_objects(frame)
        frame = detect_lanes(frame)

        # Display FPS and instructions
        cv2.putText(frame, f"FPS: {fps:.2f}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(frame, "Press 'q' to exit", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # Show window
        cv2.imshow("Self-Driving AI", frame)

        # Quit on 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



0: 480x640 1 person, 328.2ms
Speed: 5.3ms preprocess, 328.2ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 219.7ms
Speed: 3.6ms preprocess, 219.7ms inference, 3.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 261.0ms
Speed: 4.4ms preprocess, 261.0ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 234.2ms
Speed: 3.7ms preprocess, 234.2ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 204.2ms
Speed: 3.3ms preprocess, 204.2ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 197.6ms
Speed: 3.6ms preprocess, 197.6ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 250.4ms
Speed: 4.5ms preprocess, 250.4ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 244.9ms
Speed: 4.1ms preprocess, 244.9ms inference, 3.9ms postprocess per image at

Exception in thread Thread-1712 (speak_thread):
Traceback (most recent call last):
  File "C:\Users\jagda\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\jagda\anaconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jagda\AppData\Local\Temp\ipykernel_6812\528868040.py", line 32, in speak_thread
  File "C:\Users\jagda\anaconda3\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started



0: 480x640 1 cat, 276.1ms
Speed: 3.6ms preprocess, 276.1ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 640)


Exception in thread Thread-1713 (speak_thread):
Traceback (most recent call last):
  File "C:\Users\jagda\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\jagda\anaconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jagda\AppData\Local\Temp\ipykernel_6812\528868040.py", line 32, in speak_thread
  File "C:\Users\jagda\anaconda3\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started



0: 480x640 2 persons, 1 cat, 1 tie, 245.6ms
Speed: 3.7ms preprocess, 245.6ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)


...


0: 480x640 1 person, 1 train, 279.9ms
Speed: 4.3ms preprocess, 279.9ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 train, 288.8ms
Speed: 4.3ms preprocess, 288.8ms inference, 3.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 train, 1 tie, 270.2ms
Speed: 3.1ms preprocess, 270.2ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)


Exception in thread Thread-1720 (speak_thread):
Traceback (most recent call last):
  File "C:\Users\jagda\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\jagda\anaconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jagda\AppData\Local\Temp\ipykernel_6812\528868040.py", line 32, in speak_thread
  File "C:\Users\jagda\anaconda3\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started



0: 480x640 1 person, 1 train, 1 tie, 226.5ms
Speed: 3.3ms preprocess, 226.5ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 train, 1 tie, 216.3ms
Speed: 3.0ms preprocess, 216.3ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 train, 215.3ms
Speed: 3.5ms preprocess, 215.3ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 298.6ms
Speed: 3.3ms preprocess, 298.6ms inference, 3.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 train, 1 tie, 263.2ms
Speed: 2.9ms preprocess, 263.2ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 train, 225.1ms
Speed: 4.0ms preprocess, 225.1ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 train, 233.7ms
Speed: 3.6ms preprocess, 233.7ms inference, 3.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 train

Exception in thread Thread-1723 (speak_thread):
Traceback (most recent call last):
  File "C:\Users\jagda\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\jagda\anaconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jagda\AppData\Local\Temp\ipykernel_6812\528868040.py", line 32, in speak_thread
  File "C:\Users\jagda\anaconda3\Lib\site-packages\pyttsx3\engine.py", line 180, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started



0: 480x640 1 person, 1 tie, 263.7ms
Speed: 6.6ms preprocess, 263.7ms inference, 5.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 3 ties, 269.1ms
Speed: 4.1ms preprocess, 269.1ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tie, 223.9ms
Speed: 4.5ms preprocess, 223.9ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tie, 277.6ms
Speed: 3.7ms preprocess, 277.6ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 267.3ms
Speed: 4.0ms preprocess, 267.3ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 270.6ms
Speed: 3.1ms preprocess, 270.6ms inference, 3.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tie, 271.6ms
Speed: 4.0ms preprocess, 271.6ms inference, 3.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 tie, 304.4ms
Speed: 3.0ms preprocess, 304.4m